In [1]:
import datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

%matplotlib inline

pd.options.display.max_rows = 1000

In [2]:
df = pd.read_csv('./data/sample_set.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 286500 entries, 0 to 286499
Data columns (total 18 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   artist         228108 non-null  object 
 1   auth           286500 non-null  object 
 2   firstName      278154 non-null  object 
 3   gender         278154 non-null  object 
 4   itemInSession  286500 non-null  int64  
 5   lastName       278154 non-null  object 
 6   length         228108 non-null  float64
 7   level          286500 non-null  object 
 8   location       278154 non-null  object 
 9   method         286500 non-null  object 
 10  page           286500 non-null  object 
 11  registration   278154 non-null  float64
 12  sessionId      286500 non-null  int64  
 13  song           228108 non-null  object 
 14  status         286500 non-null  int64  
 15  ts             286500 non-null  int64  
 16  userAgent      278154 non-null  object 
 17  userId         278154 non-nul

In [4]:
df.duplicated().any()

False

In [5]:
df['registration'] = pd.to_datetime(df['registration'] / 1000, unit='s')
df['ts'] = pd.to_datetime(df['ts'] / 1000, unit='s')

In [6]:
for col in df.columns:
    print('Column:', col)
    print(df[col].value_counts(), '\n')

Column: artist
Kings Of Leon             1841
Coldplay                  1813
Florence + The Machine    1236
Dwight Yoakam             1135
BjÃÂ¶rk                  1133
                          ... 
Vanessa Daou                 1
Vicious                      1
Darqwan                      1
Sha Na Na                    1
Tennis                       1
Name: artist, Length: 17655, dtype: int64 

Column: auth
Logged In     278102
Logged Out      8249
Guest             97
Cancelled         52
Name: auth, dtype: int64 

Column: firstName
Payton        9632
Riley         7970
Lucero        6880
Emilia        5732
Emma          5478
Joseph        5209
Colin         4989
Nicole        4825
Joshua        4654
Brayden       4621
Alyssia       4428
Jayden        4268
Dillyn        4257
Sophia        4246
Bryson        4172
Katiana       3769
Oliver        3657
Kaleb         3603
Samuel        3486
Alexi         3437
Sawyer        3385
Michael       3382
Micah         3191
Daelin        3102
Ka

Los Angeles-Long Beach-Anaheim, CA               30131
New York-Newark-Jersey City, NY-NJ-PA            23684
Boston-Cambridge-Newton, MA-NH                   13873
Houston-The Woodlands-Sugar Land, TX              9499
Charlotte-Concord-Gastonia, NC-SC                 7780
Dallas-Fort Worth-Arlington, TX                   7605
Louisville/Jefferson County, KY-IN                6880
Philadelphia-Camden-Wilmington, PA-NJ-DE-MD       5890
Chicago-Naperville-Elgin, IL-IN-WI                5114
St. Louis, MO-IL                                  4858
Phoenix-Mesa-Scottsdale, AZ                       4846
Vineland-Bridgeton, NJ                            4825
Wilson, NC                                        4659
Denver-Aurora-Lakewood, CO                        4453
Ionia, MI                                         4428
San Antonio-New Braunfels, TX                     4373
Danville, VA                                      4257
Atlanta-Sandy Springs-Roswell, GA                 4236
New Haven-

"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"                                22751
"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.77.4 (KHTML, like Gecko) Version/7.0.5 Safari/537.77.4"                        19611
"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"                     18448
"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"                     17348
Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) Gecko/20100101 Firefox/31.0                                                                       16700
"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"                                15395
"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/37.0.2062.103 Safari/53

In [7]:
# Find customers who cancelled their subscriptions.
canc_id = list(df.loc[df['page'] == "Cancel"]['userId'])
canc = df[df['userId'].isin(canc_id)]

# Find customers who did not cancel their subscriptions.
cust = df[~df['userId'].isin(canc_id)]

## Do customers who cancel their subscriptions listen to specific artists?

In [8]:
cust['artist'].value_counts()

Coldplay                            1552
Kings Of Leon                       1542
Florence + The Machine              1048
BjÃÂ¶rk                             962
Dwight Yoakam                        952
                                    ... 
Alton Ellis                            1
Black Joe Lewis & The Honeybears       1
Hurricane Chris                        1
The Comfies                            1
Fabolous / Lil Wayne                   1
Name: artist, Length: 16652, dtype: int64

In [9]:
canc['artist'].value_counts()

Kings Of Leon             299
Coldplay                  261
Florence + The Machine    188
The Black Keys            185
Dwight Yoakam             183
                         ... 
DAN SEALS                   1
Franz Nicolay               1
Fish                        1
Gun Club                    1
Tennis                      1
Name: artist, Length: 8402, dtype: int64

## On average, do customers who cancel their subscriptions use the service for a shorter time?

In [10]:
cust['length'].mean()

249.20913538881427

In [11]:
canc['length'].mean()

248.63279564406366

## On average, after how many days do customers cancel their subscriptions?

In [12]:
canc_end_dates = canc.loc[canc['page'] == 'Cancel'][['userId', 'ts']]

In [13]:
canc_start_dates = canc.groupby('userId')['registration'].last()

In [14]:
canc_durs = pd.merge(canc_start_dates, canc_end_dates, how='inner', on='userId', right_index=True)

In [15]:
canc_durs['con_duration'] = canc_durs['ts'] - canc_durs['registration']

In [16]:
canc_durs['con_duration'].mean()

Timedelta('57 days 07:19:33.288461')

In [17]:
cust_start_dates = cust.groupby('userId')['registration'].last()

In [18]:
cust_end_dates = cust.groupby('userId')['ts'].last()

In [19]:
cust_durs = pd.merge(cust_start_dates, cust_end_dates, how='inner', on='userId', right_index=True)

In [20]:
cust_durs['con_duration'] = cust_durs['ts'] - cust_durs['registration']

In [21]:
cust_durs['con_duration'].mean()

Timedelta('86 days 14:53:41.514450')

In [22]:
cust.level.value_counts()

paid    195686
free     45950
Name: level, dtype: int64

In [23]:
canc.level.value_counts()

paid    32476
free    12388
Name: level, dtype: int64

## Do customers who cancel their subscriptions have a lower CTR?

In [24]:
ctr_cust = cust.groupby('userId')['page'].count().mean()

In [25]:
ctr_canc = canc.groupby('userId')['page'].count().mean()

## Does it matter whether customers experience a 404 error?

In [26]:
len(cust.loc[cust['status'] == 404]) / len(cust)

0.0009352910990084259

In [27]:
len(canc.loc[canc['status'] == 404]) / len(canc)

0.0007132667617689016

## What do customers do before they cancel their subs?